In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_GrgVm1


#### 1. CommaSeparatedListOutputParser

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

print(format_instructions)


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [8]:

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    temperature=0.7
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | llm | output_parser

#"AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print("AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "../data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")

AI 관련 기술 목록:
['Invalid operation: the request does not specify a valid Pandas DataFrame operation.']
 '../data/ai_technologies.csv' 파일로 저장 완료!


#### 2. JsonOutputParser

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

print(parser.get_format_instructions())

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# OpenAI 모델 설정
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


Return a JSON object.
[
    {
        "mission_name": "뉴호라이즌스",
        "goal": "명왕성 탐사",
        "agency": "나사"
    },
    {
        "mission_name": "카시니-호이겐스",
        "goal": "토성의 위성 타이탄 탐사",
        "agency": "나사"
    },
    {
        "mission_name": "창어 4호",
        "goal": "달의 뒷면 탐사",
        "agency": "중국 국가우주국"
    }
]


#### 3. PandasDataFrameOutputParser

In [ ]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('../data/titanic.csv')

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)

# 체인 생성
chain = prompt | llm | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

        # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")



Format Instructions:
 The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2,

In [11]:

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)

# 체인 생성
chain = prompt | llm | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

        # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

    print("Fare 칼럼의 평균 값 출력")
    df_query3 = "show me the average value of the Fare column"

    parser_output3 = chain.invoke({"query": df_query3})
    print(parser_output3)

except Exception as e:
    print(f"오류 발생: {e}")



Name 컬럼 출력
<class 'dict'>
{'Name': 0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object}
첫번째 행 출력
{'0': PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                               22.0
SibSp                                1
Parch     

In [14]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 응답 스키마 정의 [{},{},{}]
response_schemas = [
    #ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
    ResponseSchema(name="data", description="테이블 행을 나타내는 딕셔너리들의 리스트"),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"data": string  // 테이블 행을 나타내는 딕셔너리들의 리스트
}
```


In [15]:

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    당신은 테이블 형태의 데이터를 생성하는 AI 어시스턴트입니다(You are an AI assistant that generates tabular data).
    반드시 다음 스키마를 따르는 JSON 형식으로 데이터를 반환해야 합니다(You must return the data in JSON format that follows this schema):

    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 생성 (프롬프트 → 모델 → StructuredOutputParser)
chain = prompt | llm | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f" 오류 발생: {e}")
        return None

In [17]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    #"Create a dataset of the average apartment sale prices in Seoul for the first half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
    "2024년 상반기 서울 아파트 평균 매매가격 데이터셋을 생성해주세요. 컬럼은 다음과 같습니다: 구(District), 평균가격_억원(Average_Price_100M_KRW), 거래건수(Transactions), 전년동기대비증감률_퍼센트(YoY_Change_Percent). 서울의 25개 구에 대한 현실적인 데이터를 생성해주세요."
) 
df_seoul_housing


2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성

 Generated DataFrame:



,District,Average_Price_100M_KRW,Transactions,YoY_Change_Percent
0,강남구,42.3,1850,-2.1
1,서초구,38.7,1620,-1.8
2,용산구,35.2,980,-0.5
3,송파구,32.5,1450,-1.2
4,강동구,29.8,1100,0.3
5,마포구,28.4,1050,-0.7
6,성동구,27.9,920,-0.9
7,동작구,26.5,880,0.1
8,광진구,25.7,790,-0.4
9,강북구,24.3,650,1.2


In [18]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    #"Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
    "2024년 서울 지하철 승객 이용량이 가장 많은 상위 10개 역의 데이터셋을 생성해주세요. 컬럼은 다음과 같습니다: 역명(Station_Name), 호선(Line_Number), 일평균승객수_만명(Daily_Passengers_10K), 평일대주말비율(Weekday_Weekend_Ratio). 실제 서울 지하철의 주요 역들을 기반으로 현실적인 데이터를 생성해주세요."
)

#if df_seoul_subway is not None:
df_seoul_subway.head()


2024년 서울 지하철역별 유동 인구 데이터

 Generated DataFrame:



,Station_Name,Line_Number,Daily_Passengers_10K,Weekday_Weekend_Ratio
0,강남,2호선,19.8,1.4:1
1,서울,1·4호선,18.5,1.5:1
2,신도림,1·2호선,17.2,1.3:1
3,홍대입구,2·6호선,16.9,1.1:1
4,건대입구,2·7호선,15.7,1.2:1


In [ ]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    #"Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
    "2024년 한국의 편의점 브랜드 상위 5개사 데이터셋을 생성해주세요. 컬럼은 다음과 같습니다: 브랜드명(Brand_Name), 점포수(Store_Count), 총매출_조원(Revenue_Trillion_KRW), 시장점유율_퍼센트(Market_Share_Percent). CU, GS25, 세븐일레븐, 이마트24, 미니스톱 등 실제 한국 편의점 브랜드를 기반으로 현실적인 데이터를 생성해주세요."
)
df_korean_convenience_stores.head()